In [245]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [246]:
df = pd.read_csv('titanic_data.csv')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
885,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
886,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
887,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [247]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            176
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           686
embark_town      2
alive            0
alone            0
dtype: int64

In [248]:
df = df.drop(["alive", "sibsp", "parch", "deck", "sex", "class", "embark_town", 'who'], axis=1)
df

,survived,pclass,age,fare,embarked,adult_male,alone
0,0,3,22.0,7.2500,S,True,False
1,1,1,38.0,71.2833,C,False,False
2,1,3,26.0,7.9250,S,False,True
3,1,1,35.0,53.1000,S,False,False
4,0,3,35.0,8.0500,S,True,True
...,...,...,...,...,...,...,...
884,0,2,27.0,13.0000,S,True,True
885,1,1,19.0,30.0000,S,False,True
886,0,3,NaN,23.4500,S,False,False
887,1,1,26.0,30.0000,C,True,True


In [249]:
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col] = df[col].fillna(df[col].median())


In [250]:
mapping = {'C': 0, 'Q': 1, 'S': 2}
df['embarked'] = df['embarked'].map(mapping)
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])

df['adult_male'] = df['adult_male'].astype(int)
df['alone'] = df['alone'].astype(int)

df['pclass'] = df['pclass'].fillna(df['pclass'].mode()[0])

In [251]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

In [252]:
x = df.drop('survived', axis=1)
y = df['survived']

In [253]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [254]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [255]:
model = Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [256]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [257]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 8)              │            56 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65 (260.00 B)

 Trainable params: 65 (260.00 B)

 Non-trainable params: 0 (0.00 B)

In [258]:
model_data = model.fit(x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2968 - loss: 0.8943  
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3038 - loss: 0.8214 
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4515 - loss: 0.7647 
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5443 - loss: 0.7214 
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5570 - loss: 0.6904 
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5977 - loss: 0.6642 
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6259 - loss: 0.6434 
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6456 - loss: 0.6266 
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6568 - loss: 0.6114 
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6681 - loss: 0.5990 
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6821 - loss: 0.5880 
Epoch 12/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

In [259]:
loss, accuracy = model.evaluate(x_test, y_test)
accuracy

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8427 - loss: 0.4119 


0.8426966071128845